In [ ]:
def prepare_features(event):
    # Extract meaningful features from the raw XML or parsed fields
    features = {
        "msg_type": event["msg_type"],
        # Add more features based on Darwin schema
    }
    return features

def lambda_handler(event, context):
    predictions_events = []

    for record in event['Records']:
        decoded_data = base64.b64decode(record['kinesis']['data']).decode('utf-8')
        rail_event = json.loads(decoded_data)

        features = prepare_features(rail_event)
        prediction = predict([features])  # ensure it's wrapped in a list

        output = {
            'model': 'rail_event_classifier',
            'prediction': prediction,
            'event_type': rail_event.get("msg_type"),
            'ts': rail_event.get("ts")
        }

        kinesis_client.put_record(
            StreamName=os.getenv('PREDICTIONS_STREAM_NAME'),
            Data=json.dumps(output),
            PartitionKey=rail_event.get("ts", "default")
        )

        predictions_events.append(output)

    return {"predictions": predictions_events}
